# Given a dataframe of contaminations, search the Alaska Satellite Facility for matching satellite imagery results 

**⭐ Connect to the ASF API via your username and password**

In [1]:
import getpass
import asf_search as asf

username = input('Username:')
password = getpass.getpass('Password:')

try: 
    user_pass_session = asf.ASFSession().auth_with_creds(username, password)
except asf.ASFAuthenticationError as e:
    print(f'Auth Failed: {e}')
else:
    print('Success!')

session=asf.ASFSession()

Username: aglevy15
Password: ········


Success!


**⭐ For a given dataframe, search ASF for matching images based on spill date + 4 days, latitute and longitude**

**Create a dictionary that returns all contaminations searched with some key high-level identifying features and their ASF results. ASF results includes a scene ID which is searchable in Vertex, an RBG JPG and a ZIP file link to download the entirety of the ASF search results**

In [2]:
import asf_search.constants as asf_constants
import warnings
import datetime
from datetime import timedelta
import time
import random
from requests.exceptions import RequestException
import pandas as pd

def search_asf_images(df, spill_id = 'Spill ID', start_date='Spill Date', end_date='Case Closed Date', lat_col='Latitude', lon_col='Longitude'):
    
    #surpress this specific warning as we know our datetimes are correctly formatted
    warnings.filterwarnings("ignore", message="Parsing dates involving a day of month without a year specified")
    
    asf_constants.INTERNAL.CMR_TIMEOUT = 60 
    results_with_url = []
    max_retries=3

    #loop through the dataframe to get values to search for in ASF
    for _, row in df.iterrows():
        results= []
        retries = 0
        while retries < max_retries:
            try:
                lat = float(row[lat_col])
                lon = float(row[lon_col])
                start = pd.to_datetime(row[start_date], errors='coerce')
                end = start + pd.Timedelta(days=4) 
                
                #if using end_date instead of start+1, check if end is empty and if so, default to 1 day after start 
                #end = pd.to_datetime(row[end_date])  
                #if pd.isna(end):
                   #end = start + pd.Timedelta(days=1)
                
                aoi = f"POINT({lon} {lat})"
             
                # Query ASF API
                opts = {
                    'platform': asf.PLATFORM.SENTINEL1,
                    'processingLevel': asf.PRODUCT_TYPE.GRD_HD,
                    'start': start,
                    'end': end
                }
                results=asf.geo_search(intersectsWith=aoi, **opts) 
                
                break #success, exist retry loop
                                
            except Exception as e:
                if "CMR" in str(e) or "timeout" in str(e).lower():
                    print(f"🌐 Timeout on row {row.name}, retrying in a few seconds...")
                    time.sleep(random.randint(3, 6))
                    retries += 1
                else:
                    print(f"❌ Failed on row {row.name}: {e}")
                    break  # fail for other errors
        if retries == max_retries:
            print(f"Max retries exeeded for row {row.name}")
            continue 

        #if there are no ASF results, populate asf_results with empty
        if len(results)==0:
            results_with_url.append({
                'Spill ID': row[spill_id],
                'Spill Date': row[start_date],
                'ASF Results': []
        })
    
        #else there are multiple results for the scene ID, append to the asf results dict 
        else: 
            asf_results=[]
            for result in results:
                asf_results.append({
                    'Scene ID': result.properties['sceneName'],
                    'RBG_URL': result.properties['browse'],
                    'zip': result.properties['url']
                })
            results_with_url.append({
                'Spill ID': row[spill_id],
                'Spill Date': row[start_date],
                'ASF Results':asf_results
            })
                
    empty_count = sum(
        1 for row in results_with_url 
        if not row.get('ASF Results')  # catches [], None, ''
        )
    
    print(f"Empty ASF Results count: {empty_count}")
    #return a dictionary with all the data
    return results_with_url

**Recommendation: search only a small subset of datapoints before running a large query on ASF. You can do this by running on ocean_df.head(n) to search for n number of rows instead of the entire dataframe**

**Adding %store -r allows you to access the dataframe from notebook 1**

In [3]:
%store -r ocean_df

results = search_asf_images(ocean_df)

Empty ASF Results count: 169


**⭐ The number spills that have no ASF results will display after running!**

**To return results as a dataframe:** 
```results_df = pd.DataFrame(results)```

**To download your results into the a folder in the same directory:** 
```results_df.to_csv("asf_adec_search_results.csv", index=False)```

**Special tip: if you're unsure where the CSV is saving, run this code to figure out what directory your csv is saving to :** 
```import os print(os.getcwd())```